# DS_PROD-1. Подготовка модели к продакшену и деплой

###  Содержание <a class="anchor" id=0></a>

- [2. Сохранение и загрузка моделей: pickle и joblib](#2)

- [2.1 Библиотека Joblib](#3)

- [3. Практика: pickle](#4)

- [4. Сохранение и загрузка моделей: PMML и ONNX-ML](#5)

- [5. Деплой модели. Протоколы сетевого взаимодействия](#6)

- [6. Деплой модели. Обзор фреймворков](#7)

- [7. Пишем сервер на Flask](#8)

# 2. Сохранение и загрузка моделей: pickle и joblib <a class="anchor" id=2></a>

## 2.1 Библиотека pickle 


In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.linear_model import LinearRegression

In [20]:
from sklearn.linear_model import LinearRegression

from sklearn.datasets import load_diabetes

# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)
# Инициализируем модель линейной регрессии
regressor = LinearRegression()
# Обучаем модель
regressor.fit(X,y)

## LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

LinearRegression()

In [21]:
# Serialization (сериализация)
model = pickle.dumps(regressor)

print(type(regressor))
print(type(model))

<class 'sklearn.linear_model._base.LinearRegression'>
<class 'bytes'>


In [22]:
# De-Serialization (десериализация)

regressor_from_bytes = pickle.loads(model)
regressor_from_bytes

LinearRegression()

In [23]:
# Производим сериализацию и записываем результат в файл формата pkl
with open('myfile.pkl', 'wb') as output:
    pickle.dump(regressor, output)

In [24]:
# Производим десериализацию и извлекаем модель из файла формата pkl
with open('myfile.pkl', 'rb') as pkl_file:
    regressor_from_file = pickle.load(pkl_file)

regressor_from_file
## LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

LinearRegression()

In [25]:
# Проверяем, что все элементы массивов предсказаний совпадают между собой
print(all(regressor.predict(X) == regressor_from_bytes.predict(X)))
## True
print(all(regressor.predict(X) == regressor_from_file.predict(X)))
## True

True
True


## ОГРАНИЧЕНИЯ

Как мы упоминали, у pickle есть ограничения. Например, мы не можем сериализовать лямбда-функции. Давайте посмотрим, что нам вернёт следующий код:

In [ ]:
my_lambda = lambda x: x*2
with open('my_lambda.pkl', 'wb') as output:
    pickle.dump(my_lambda, output)
 
##"PicklingError: Can't pickle <function <lambda>"

Совет. В таких случаях лучше пользоваться пакетом [dill](https://github.com/uqfoundation/dill).

In [26]:
from dill import dumps, loads

squared = lambda x: x**2
loads(dumps(squared))(3)

9

## Pipelines

In [19]:
import pickle

from sklearn.linear_model import LinearRegression

from sklearn.datasets import load_diabetes

from sklearn.feature_selection import SelectKBest, f_regression

from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import Pipeline

# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)

# Создаём пайплайн, который включает нормализацию, отбор признаков и обучение модели
pipe = Pipeline([  
  ('Scaling', MinMaxScaler()),
  ('FeatureSelection', SelectKBest(f_regression, k=5)),
  ('Linear', LinearRegression())
  ])

# Обучаем пайплайн
pipe.fit(X, y)

Pipeline(steps=[('Scaling', MinMaxScaler()),
                ('FeatureSelection',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x000001BF53058F70>)),
                ('Linear', LinearRegression())])

In [27]:
# Сериализуем pipeline и записываем результат в файл
with open('my_pipeline.pkl', 'wb') as output:
    pickle.dump(pipe, output)

In [28]:
# Десериализуем pipeline из файла
with open('my_pipeline.pkl', 'rb') as pkl_file:
    loaded_pipe = pickle.load(pkl_file)

In [29]:
# Сравниваем предсказания исходного и восстановленного пайплайнов
print(all(pipe.predict(X) == loaded_pipe.predict(X)))

## True

True


>Примечание. Если мы хотим сохранять сериализованные пайплайны в виде потока байтов, нужно использовать функции `dumps()` и `loads()`, а не `dump()` и `load()`.

Однако в процессе предобработки могут возникнуть шаги, которые нельзя реализовать стандартными методами `sklearn`. Например, для решения многих задач в нашем курсе мы часто использовали `feature engineering`, чтобы повысить качество работы моделей. Как встроить этот шаг в исходный пайплайн?

Для этого в `sklearn` можно организовать так называемые кастомные трансформеры. Такой трансформер должен наследоваться от двух классов: `TransformerMixin` и `BaseEstimator`.

Посмотрим на шаблон кастомного трансформера:

In [30]:
from sklearn.base import TransformerMixin, BaseEstimator
class MyTransformer(TransformerMixin, BaseEstimator):
    '''Шаблон кастомного трансформера'''
 
    def __init__(self):
        '''
        Здесь прописывается инициализация параметров, не зависящих от данных.
        '''
        pass
 
    def fit(self, X, y=None):
        '''
        Здесь прописывается «обучение» трансформера.
        Вычисляются необходимые для работы трансформера параметры (если они нужны).
        '''

        return self
 
    def transform(self, X):
        '''
        Здесь прописываются действия с данными.
        '''
        return X

У трансформера должно быть три обязательных метода:

* `__init__()` — метод, который вызывается при создании объекта данного класса. Он предназначен для инициализации исходных параметров.
Например, у трансформера для создания полиномиальных признаков `PolynomialFeatures` из `sklearn` в методе `__init__()` параметр `degree` задаёт степень полинома.

* `fit()` — метод, который вызывается для «обучения» трансформера. Он должен возвращать ссылку на сам объект (`self`).
Например, в трансформере `StandardScaler` в методе `fit()` прописано вычисление среднего значения и стандартного отклонения в каждом столбце таблицы, переданной в качестве параметра метода `fit()`.

* `transform()` — метод, который трансформирует приходящие на вход данные. Он должен возвращать преобразованный массив данных.
Например, при вызове метода `transform()` у `StandardScaler` из `sklearn` внутри происходит преобразование — вычитание из каждого столбца среднего и деление результата на стандартное отклонение. Причём среднее и стандартное отклонение вычисляются заранее в методе `fit()`.

>Примечание. Как мы знаем, у некоторых трансформеров из `sklearn`, например у того же `MinMaxScaler`, есть ещё и метод `fit_transform()`, который является комбинацией методов `fit()` и `transform()`.

Наш трансформер пока что ничего не делает. Предположим, мы хотим генерировать в данных новый признак, который является простым произведением первых трёх столбцов таблицы. Давайте пропишем в методе `transform()` эти действия.

Для работы такого трансформера нужны только исходные данные без дополнительных параметров, поэтому методы `__init__()` и `fit()` остаются без изменений.

In [31]:
class MyTransformer(TransformerMixin, BaseEstimator):
    '''Шаблон кастомного трансформера'''


    def __init__(self):
        '''Здесь прописывается инициализация параметров, не зависящих от данных.'''
        pass


    def fit(self, X, y=None):
        '''
        Здесь прописывается «обучение» трансформера.
        Вычисляются необходимые для работы трансформера параметры (если они нужны).
        '''
        return self


    def transform(self, X):
        '''Здесь прописываются действия с данными.'''
        # Создаём новый столбец как произведение первых трёх
        new_column = X[:, 0] * X[:, 1] * X[:, 2]
        # Для добавления столбца в массив нужно изменить его размер на (n_rows, 1)
        new_column = new_column.reshape(X.shape[0], 1)
        # Добавляем столбец в матрицу измерений
        X = np.append(X, new_column, axis=1)
        return X

Посмотрим, как работает наш кастомный трансформер. Создадим объект трансформера, вызовем метод transform и посмотрим на результирующий размер таблицы.

In [32]:
# Инициализируем объект класса MyTransformer (вызывается метод __init__)
custom_transformer = MyTransformer()
# Чисто формально вызываем метод fit, но у нас он ничего не делает
custom_transformer.fit(X)
# Трансформируем исходные данные (вызывается метод transform)
X_transformed = custom_transformer.transform(X)
print('Shape before transform: {}'.format(X.shape))
print('Shape after transform: {}'.format(X_transformed.shape))

## Shape before transform: (442, 10)
## Shape after transform: (442, 11)

Shape before transform: (442, 10)
Shape after transform: (442, 11)


Видно, что в результате трансформации в исходную матрицу наблюдений добавился новый столбец.

Теперь давайте встроим этот трансформер в сам пайплайн — для этого достаточно добавить новый шаг в пайплайн.

In [33]:
# Создаём пайплайн, который включает Feature Engineering, нормализацию, отбор признаков и обучение модели
pipe = Pipeline([  
  ('FeatureEngineering', MyTransformer()),
  ('Scaling', MinMaxScaler()),
  ('FeatureSelection', SelectKBest(f_regression, k=5)),
  ('Linear', LinearRegression())
  ])

# Обучаем пайплайн
pipe.fit(X, y)

Pipeline(steps=[('FeatureEngineering', MyTransformer()),
                ('Scaling', MinMaxScaler()),
                ('FeatureSelection',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x000001BF53058F70>)),
                ('Linear', LinearRegression())])

Наконец можно сериализовать полученный `pipeline`:

In [34]:
# Сериализуем pipeline и записываем результат в файл
with open('my_new_pipeline.pkl', 'wb') as output:
    pickle.dump(pipe, output)

In [36]:
# Задание 2.5

# Десериализуйте полученный pipeline с добавленным в него кастомной трансформации из файла. 
# Затем предскажите значение целевой переменной для наблюдения, которое описывается следующим вектором:

features = np.array([[ 0.00538306, -0.04464164,  0.05954058, -0.05616605,  0.02457414, 0.05286081, -0.04340085,  0.05091436, -0.00421986, -0.03007245]])

# Десериализуем pipeline из файла
with open('my_new_pipeline.pkl', 'rb') as pkl_file:
    loaded_pipe = pickle.load(pkl_file)
    
loaded_pipe.predict(features)

array([173.01985747])

# Библиотке Joblib  <a class="anchor" id=3></a>

[к содержанию](#0)

Как мы видим, `pickle` прекрасно справляется со своей задачей: мы можем сериализовать и восстанавливать любые `Python`-объекты, включая модели и даже пайплайны. Однако иногда массивы данных, на которых обучаются модели, бывают настолько большими, что после загрузки из `pickle` невозможно восстановить объект полностью.

В таких случаях вместо `pickle` лучше использовать библиотеку `joblib`. Этот модуль более эффективен и надёжен для работы с объектами, которые содержат большие массивы данных. Пожалуй, единственный минус этого модуля в том, что он может «консервировать» только в файл, поэтому вы не сможете получить объект в виде бинарной строки и работать с ним. В модуле попросту отсутствуют методы для работы с бинарной строкой. Формат файлов для сохранения — `.joblib`.

В остальном работа с `joblib` полностью идентична работе с `pickle`: после обучения модели производим сериализацию с помощью функции `dump()`, а в коде самого приложения, где нужно использовать модель, выполняем десериализацию с помощью функции `load()`. В каждую из этих функций необходимо передать путь до файла для записи и чтения соответственно.

Для иллюстрации работы сохраним полученную линейную регрессию:

In [38]:
import joblib

import joblib

# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)
# Обучаем модель линейной регрессии
regressor = LinearRegression()
regressor.fit(X, y)
# Производим сериализацию и сохраняем результат в файл формата .joblib
joblib.dump(regressor, 'regr.joblib')

## ['regr.joblib']

['regr.joblib']

In [39]:
# Десериализуем модель из файла
clf_from_jobliv = joblib.load('regr.joblib') 
# Сравниваем предсказания
all(regressor.predict(X) == clf_from_jobliv.predict(X))

## True

True

# 3. Практика: pickle <a class="anchor" id=4></a>

[к содержанию](#0)



In [2]:
# Задание 3.1 При загрузке вывелся секретный код. Введите его в поле ниже.

import pickle

with open('model.pkl', 'rb') as pkl_file:
    model = pickle.load(pkl_file)
    
model

secret word: skillfactory
how is this possible? answer is here: https://youtu.be/xm-A-h9QkXg


c:\Users\AubakirovMA\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


LinearRegression(positive=True)

In [4]:
# Задание 3.3 Теперь необходимо применить модель. Сделайте предсказание для следующего набора фичей: [1, 1, 1, 0.661212487096872]. 
# Введите результат, предварительно округлив его до трёх знаков после точки-разделителя.

model.predict([[1, 1, 1, 0.661212487096872]])

array([0.666])

In [10]:
# У присланной вам модели есть два поля (атрибута) с именами a и b. Создайте из них словарь с такими же именами ключей и значениями, а затем сохраните его в файл с помощью модуля pickle.

my_dict = {'a': model.a, 'b': model.b}

# Сериализуем pipeline и записываем результат в файл
with open('my_dict.pkl', 'wb') as output:
    pickle.dump(my_dict, output)

In [11]:
!python hw1_check_ol.py my_dict.pkl

('secret code 2:', '3c508')


# 4. Сохранение и загрузка моделей: PMML и ONNX-ML <a class="anchor" id=5></a>

[к содержанию](#0)


>Среда или требования к инференсу модели для вашего проекта могут быть устроены так, что потребуют реализации на языке программирования, отличном от `Python`. Например, если компания разрабатывает десктопное приложение, то для внедрения модели её потребуется «перевести» на `Java` или `C++`. Как это сделать?

## PREDICTIVE MODEL MARKUP LANGUAGE

В таких случаях используется генерация файла формата `PMML` (`Predictive Model Markup Language`).

`PMML` — это `XML`-диалект, который применяется для описания статистических и `DS`-моделей. `PMML`-совместимые приложения позволяют легко обмениваться моделями данных между собой. Разработка и внедрение `PMML` осуществляется IT-консорциумом `Data Mining Group`.

Подробнее с `PMML` можно ознакомиться на [официальном сайте](https://dmg.org/pmml/v4-4/GeneralStructure.html).

К сожалению, далеко не все библиотеки для машинного обучения (в том числе `sklearn`) поддерживают возможность сохранения обученной модели в указанном формате. Однако для этого можно использовать сторонние библиотеки, и одной из самых популярных является [Nyoka](https://nyoka-pmml.github.io/nyoka/).

Давайте сохраним модель из предыдущего блока в формат `PMML`.

Для установки можно использовать систему управления пакетами `pip`: `pip install nyoka`

In [12]:
from nyoka import skl_to_pmml
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True)
cols = load_diabetes()['feature_names']

scaler = MinMaxScaler()
pipe = Pipeline([  
            ('Scaling', MinMaxScaler()),
            ('Linear', LinearRegression())
        ])
# Обучение пайплайна, включающего линейную модель и нормализацию признаков
pipe.fit(X, y)
# Сохраним пайплайн в формате pmml в файл pipeline.pmml
skl_to_pmml(pipeline=pipe, col_names=cols, pmml_f_name="pipeline.pmml")

Итак, мы построили пайплайн обработки данных и обучили модель линейной регрессии. После этого мы с помощью функции `skl_to_pmml` сохранили модель в файл `pipe.pmml`.

Откройте файл `pipe.pmml` с помощью любого текстового редактора.

Давайте рассмотрим этот файл подробнее:

* Секция `<DataDictionary>` содержит информацию о признаках, включая наименование и тип данных, используемых для построения модели.

* Секция `<TransformationDictionary>` содержит информацию о необходимых преобразованиях для каждого признака. Обратите внимание, что в этом блоке также содержится информация для трансформации. Так как мы использовали `minMaxScaler()`, то в файле записаны минимальное и максимальное значения.

***

# OPEN NEURAL NETWORK EXCHANGE

В разработке моделей на основе нейронных сетей сегодня наиболее распространён формат `ONNX` (`Open Neural Network Exchange`).

`ONNX` ([Open Neural Network Exchange](https://onnx.ai/)) — это открытый стандарт для обеспечения совместимости моделей машинного обучения. Он позволяет разработчикам искусственного интеллекта использовать модели с различными инфраструктурами, инструментами, средами исполнения и компиляторами.

Стандарт совместно поддерживается компаниями `Microsoft`, `Amazon`, `Facebook` и другими партнёрами как проект с открытым исходным кодом.

Часто стандарт `ONNX` и его библиотеки используют для конвертации из одного фреймворка в другой (например, из `PyTorch` в `TensorFlow` для использования в продакшене). Для конвертации различных фреймворков (не только DL) в формат ONNX и обратно существует [ряд библиотек](https://github.com/onnx):

* [ONNX-Tensorflow](https://github.com/onnx/onnx-tensorflow);
* [Tensorflow-ONNX](https://github.com/onnx/tensorflow-onnx);
* [Keras-ONNX](https://github.com/onnx/keras-onnx);
* [Sklearn-ONNX](https://github.com/onnx/sklearn-onnx).
* и другие.

Также в рамках стандарта `ONNX` есть инструмент [ONNX-runtime](https://github.com/microsoft/onnxruntime). Он служит для ускорения инференса `Python`-моделей, а также инференса на других языках, например `Java`, `C++`.

>pip install onnx
>
>pip install skl2onnx

>Для выполнения этого задания вам понадобится ознакомиться с документацией [здесь](http://onnx.ai/sklearn-onnx/api_summary.html) и [здесь](https://onnx.ai/sklearn-onnx/).

In [ ]:
import onnxruntime as rt 
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from skl2onnx import ___1___
from skl2onnx.common.data_types import ___2___


# загружаем данные
X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7)
print(X_train.shape, X_test.shape)

# обучаем модель
model = LinearRegression()
model.fit(___3___, y_train)

# делаем инференс моделью на тесте
test_pred = model.predict(___4___)
print('sklearn model predict:\n', test_pred)

# конвертируем модель в ONNX-формат
initial_type = [('float_input', ___5___([None, ___6___]))]
model_onnx = ___7___(model, initial_types=initial_type)

# сохраняем модель в файл
with open("model.onnx", "wb") as f:
	f.write(model_onnx.SerializeToString())
 	 
# Делаем инференс на тесте через ONNX-runtime
sess = rt.___8___("model.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
test_pred_onnx = sess.run([label_name],
                	{input_name:  X_test.astype(np.float32)})[0].reshape(-1)
print('onnx model predict:\n',test_pred_onnx)

# 5. Деплой модели. Протоколы сетевого взаимодействия <a class="anchor" id=6></a>

[к содержанию](#0)


## МОДЕЛИ СЕТЕВОГО ВЗАИМОДЕЙСТВИЯ

Начнём с первого вопроса и немного поговорим о том, как происходит взаимодействие между серверами по сети, то есть разберём процесс обмена информацией между компьютерами.

Наиболее известные модели сетевого взаимодействия — [OSI](https://ru.wikipedia.org/wiki/Сетевая_модель_OSI) и [TCP/IP](https://ru.wikipedia.org/wiki/TCP/IP).

<img src=DSPROD_1.png width=600>

Эти модели распределяют сетевые протоколы по разным уровням взаимодействия. Что такое протокол?

>Вообще протокол — это некоторый набор правил, определяющий принципы взаимодействия устройств в сети. В нашем случае это правила, по которым программа, получив по сети набор битов, понимает, как его прочитать и что он значит.

Для того чтобы обмен информацией между устройствами проходил успешно, все устройства (участники процесса) должны следовать условиям протокола. В сети поддержка протоколов встраивается или в аппаратную (в «железо»), или в программную часть (в код системы), или в обе этих части. 

На схеме ниже представлены примеры протоколов, а также уровни их распределения в модели `TCP/IP`:

<img src=DSPROD_2.png width=600>

В процессе сетевого взаимодействия участвуют как минимум два устройства — устройство-отправитель и устройство-получатель. Говоря простым языком, каждая из моделей сетевых взаимодействий устанавливает правила и регламенты по отправке сообщений между компьютерами.

Отправленное сообщение проходит все уровни, начиная от прикладного уровня приложений и заканчивая физическим уровнем доступа к сети. Когда сообщение доходит до адресата, оно также проходит все уровни в обратном порядке.

<img src=DSPROD_3.png width=600>

Нам как дата-сайентистам не нужно знать все подробности того, как работают и чем отличаются друг от друга разные уровни взаимодействия. Однако для общего развития и понимания того, как устройства обмениваются информацией, вы можете прочитать об уровнях моделей OSI и TCP/IP [здесь](https://selectel.ru/blog/osi-for-beginners/) и [здесь](https://zametkinapolyah.ru/servera-i-protokoly/chto-takoe-model-osi-etalonnaya-model-setevogo-vzaimodejstviya-urovni-setevoj-modeli-osi-primery-i-prostoe-obyasneniya-principa-raboty-semiurovnevoj-modeli.html#__OSI-3).

Для наших целей (деплой модели в прод) достаточно уметь работать всего с тремя протоколами:

>* `IP` — протокол сетевого уровня. Он определяет путь, по которому передаются данные.
>
>* `TCP` — соответствует транспортному уровню, а значит, определяет, как передаются данные.
>
>* `HTTP` — относится к прикладному уровню, описывающему взаимодействие приложений с сетью.

<img src=DSPROD_4.png width=600>

## `IP`

`IP` (Internet Protocol) — один из главных протоколов сетевого взаимодействия. Он отвечает за маршрутизацию трафика по сети, то есть определяет путь, по которому отправятся данные. Данные передаются пакетами (или датаграммами), которые формирует протокол IP.

Важным свойством IP является отсутствие гарантированной доставки пакетов и их цельности: пакеты могут прийти в другой очерёдности (не в той, в которой их отправляли), прийти повреждёнными (тогда они уничтожаются) или вообще не прийти.

Путь, по которому отправятся данные, строится на основе IP-адресов.

IP-адрес — это уникальный адрес, используемый для связи устройств внутри сети.

IP-адрес устроен довольно просто: чаще всего это четыре числа, разделённых точками (такой формат поддерживается в протоколе IPv4). Например, вот один из самых популярных IP-адресов — 192.168.0.1. Вы могли вводить его, чтобы зайти в настройки своего роутера.

Каждое из чисел в адресе — это восьмизначное двоичное число, или, правильнее говорить, октет. Оно может принимать значения от 0000 0000 до 1111 1111 в двоичной системе или от 0 до 255 — в десятичной системе счисления, то есть 256 разных значений.

Получается, что диапазон IP-адресов стартует с 0.0.0.0 и заканчивается 255.255.255.255. Если посчитать количество всех адресов в этом диапазоне, получится чуть больше четырёх миллиардов.

Уникальность IP-адреса может быть глобальной (в рамках всего интернета) или локальной (в рамках локальной подсети). Некоторые IP-адреса не являются общедоступными и зарезервированы для специальных целей, например диапазоны IP-адресов:

| Диапазон | Назначение |
| - | - |
| 127.0.0.1–127.255.255.255 | используются для связи внутри локальной машины (localhost) |
| 172.16.0.1–172.31.255.255 | используются для частных подсетей (недоступных из интернета) |
| 198.18.0.1–198.19.255.254 | используются для тестирования производительности |


>`localhost` — зарезервированное доменное имя для IP-адресов из диапазона 127.0.0.1–127.255.255.255 (в сети из одного компьютера — для 127.0.0.1).

В компьютерной сети `localhost` относится к компьютеру, на котором запущена программа. Компьютер работает как виртуальный сервер. Тем самым создаётся так называемая «внутренняя петля»: обращаясь по IP-адресу `localhost`, вы, по сути, заставляете компьютер общаться с самим собой (хотя на самом деле внутри всё немного сложнее). Это нужно, например, для разработки и тестирования клиент-серверных приложений на одной машине (то есть и клиент, и сервер находятся на одном компьютере), что позволяет при разработке не использовать сетевое оборудование, дополнительные программные модули и тому подобное.

## `TCP`

`TCP` (`Transmission Control Protocol`) — протокол транспортного уровня. Он отвечает за управление передачей данных и гарантирует:

* доставку пакетов (посылает пакеты повторно, если они не были доставлены);

* последовательность и целостность доставки пакетов (используя нумерацию и контрольные суммы для проверки);

* устраняет дубликаты в случае необходимости.

Важной особенностью TCP является то, что перед отправкой данных он «устанавливает соединение» с получателем — обменивается управляющей информацией. После отправки пакетов источник ждёт подтверждения от получателя, что пакеты были доставлены.

Обычно на одном узле сети (сервере, компьютере) работают несколько приложений/процессов одновременно. Для идентификации приложения на источнике и получателе используется порт, который задаётся целым неотрицательным числом. Процесс или приложение могут зарезервировать у ОС определённый порт, например, для передачи данных по сети.

Порты разделяют на системные (0–1023), и пользовательские (1024–49151). Некоторые номера портов определены для конкретных приложений, например:

* 22 — протокол SSH для безопасной передачи данных;

* 25 — протокол SMTP для незащищённой передачи e-mail-сообщений;

* 80 — протокол HTTP.

## `HTTP`

`HTTP` — это наиболее широко используемый протокол. Все сайты, на которые вы заходите, работают по этому протоколу. Он был разработан именно для передачи содержимого `HTML`-страниц в интернете, но впоследствии стал использоваться и для других целей. Например, `HTTP` применяется для налаживания взаимодействия между сервисами в сложных системах. Этим он нам и интересен.

Итак, `HTTP` — это протокол, который работает по принципу клиент-сервер.

<img src=DSPROD_5.png width=600>

Это означает, что во взаимодействии участвует две программы, причём в разных ролях. Одна из них — клиент, или «заказчик услуг», формирует запрос и отправляет его к серверу. Сервер, или «поставщик услуг», получив запрос, обрабатывает его, формирует ответ и возвращает его клиенту.

## СТРУКТУРА HTTP-ЗАПРОСОВ

<img src=DSPROD_6.png width=600>

Запрос и ответ в HTTP являются строками, составленными в соответствии с протоколом.

Запрос состоит из трёх частей.

<img src=DSPROD_7.png width=600>

* Стартовая строка, или `Request Line` — по ней определяется вид запроса.

* Заголовки запроса, или `Request Headers` — дополнительные параметры запроса, в которых обычно передаётся служебная информация, например, в каком формате ожидается ответ или информация о клиенте.

* Тело запроса, или `Request Message Body` — содержит данные для передачи. Эта часть присутствует не всегда.

Давайте чуть подробнее разберём первую часть.

Первое, что указано в стартовой строке — это метод, или тип запроса. Есть набор стандартных методов, но теоретически вы можете создавать и свои.

## Основные методы:

* `GET` — обычно означает получение содержимого ресурса и не содержит тела.

* `POST` — наоборот, передача данных ресурсу.

* `PUT` — обновление ресурса.

* `DELETE` — удаление ресурса.

В `HTTP` мы работаем с ресурсами, которые расположены по некоторому адресу на сервере. Изначально под ресурсами понимались `HTML`-файлы на сайте (вёрстка сайта), но сейчас это уже некоторое абстрактное понятие.

Адрес ресурса, или `URI` (`Uniform Resource Identifier`) — это то, что вы видите в адресной строке браузера. Он следует за методом в стартовой строке запроса.


## Чем отличаются URI, URL и URN?
 
Для начала расшифруем аббревиатуры:

`URI` — `Uniform Resource Identifier` (унифицированный идентификатор ресурса);
`URL` — `Uniform Resource Locator` (унифицированный определитель местонахождения ресурса);
`URN` — `Unifrorm Resource Name` (унифицированное имя ресурса).

Большинство считает, что http://google.com или https://skillfactory.ru/ — это просто URL-адреса. Тем не менее, мы можем говорить о них как о URI. URI представляет собой комбинацию URL-адресов и URN. Таким образом, мы можем с уверенностью сказать, что все URL являются URI. Однако обратное неверно.

>Давайте рассмотрим это на простом примере. 
>
>Имя «Джон Сноу» — это URN. Место, в котором живёт Джон, например «Улица Вестерос, 13» — это уже URL. Вас можно идентифицировать как уникальное лицо с вашим именем или вашим адресом. Эта уникальная личность — это уже URI. И хотя ваше имя может быть вашим уникальным идентификатором (URI), оно не может быть URL-адресом, поскольку имя не позволяет найти местоположение.
>
>Теперь дадим расширенное определение терминам:
>
>URI — имя и адрес ресурса в сети. URI включает в себя URL и URN.
>
>Пример: https://wiki.merionet.ru/images/vse-chto-vam-nuzhno-znat-pro-devops/1.png
>
>URL — адрес ресурса в сети. URL определяет местонахождение и способ обращения к нему. 
>
>Пример: https://wiki.merionet.ru
>
>URN — имя ресурса в сети. URN определяет только название ресурса, но не говорит, как к нему подключиться.
>
>Пример: images/vse-chto-vam-nuzhno-znat-pro-devops/1.png

Последней идёт версия HTTP-протокола (кстати, последняя актуальная версия, 1.1, появилась ещё в 1999 году).

<img src=DSPROD_8.png width=600>

Ответ также состоит из стартовой строки, заголовков и тела:

<img src=DSPROD_9.png width=600>

>Основное отличие — в стартовой строке: там вместо метода и URI указывается код состояния. Это численное значение, которое показывает результат обработки. Коды задаются протоколом, и вы наверняка встречали «ошибку 404» на сайтах.

`404` — это как раз код состояния, означающий, что ресурса с заданным URI не существует на сервере.

Группы кодов состояния ответа HTTP-сервера делятся на следующие группы:

* информационные (100–199);
* успешно (200–299);
* перенаправление (300–399);
* ошибка клиента (400–499);
* ошибка сервера (500–599).

*** 
## REST (REPRESENTATIONAL STATE TRANSFER)

<img src=DSPROD_10.png width=600>

Cпецификация `HTTP` не обязывает сервер понимать все методы, а также не указывает серверу, что он должен делать при получении запроса с тем или иным методом. Поэтому был изобретён архитектурный стиль `REST`.

Он даёт более верхнеуровневые указания, чем `HTTP`-протокол, а именно:

* как правильно организовывать адресацию к ресурсам;
* какие методы у этих ресурсов должны быть;
* какой ожидается результат.

>Основная концепция философии `REST` заключается в том, что клиентом `RESTful`-сервера может быть что/кто угодно: браузер, другое приложение, разработчик. Веб-приложение, спроектированное по правилам `REST`, предоставляет информацию о себе в форме информации о своих ресурсах.

Ресурс может быть любым объектом, о котором сервер предоставляет информацию. Например, в `API Instagram` ресурсом может быть пользователь, фотография, хэштег. Каждая единица информации (ресурс) однозначно определяется URL.

* GET-запрос `/rest/users` — получение информации обо всех пользователях.
* GET-запрос `/rest/users/125` — получение информации о пользователе с id=125.
* POST-запрос `/rest/users` — добавление нового пользователя.
* PUT-запрос `/rest/users/125` — изменение информации о пользователе с id=125.
* DELETE-запрос `/rest/users/125` — удаление пользователя с id=125.

Поскольку спецификация REST является общепризнанной и широко распространённой, то следовать ей очень полезно.

Если вы говорите, что ваш сервис `RESTful` (то есть построен по этой спецификации), то любой разработчик сразу поймёт, как примерно устроен сервис, а значит, сможет быстро начать им пользоваться: отправлять правильные запросы и получать тот результат, который ожидает.

# 6. Деплой модели. Обзор фреймворков <a class="anchor" id=7></a>

[к содержанию](#0)

## FLASK

`Flask` позиционируется как микрофреймворк для написания легковесных сервисов, но при этом он достаточно гибкий и кастомизируемый, что позволяет использовать его в проектах любой сложности. Как и все остальные фреймворки, он работает на основе маршрутизации запросов, приходящих на сервер, который и должен обработать эти запросы.

<img src=DSPROD_11.png width=600>

В случае с `Flask` маршрутизация выглядит как обычные функции `Python`, которые вы помечаете специальным декоратором `@app.route`. Он связывает эту функцию с адресом, на который приходит запрос. В этой функции вы должны разобрать параметры запроса, возможно, выполнить какую-то логику и вернуть ответ клиенту.

Таким образом, для написания минимально функционального сервера достаточно добавить в скрипт всего несколько строк: необходимые импорты, создание `Flask`-приложения, декоратор, который свяжет адрес (`endpoint`, эндпоинт) с функцией, и запуск приложения. Последнее как раз и запустит работу `Flask`: он будет слушать запросы, маршрутизировать их и возвращать ответ.

>Рассмотрим на примере. Сайт почты `Mail.ru` имеет адрес https://e.mail.ru. Для того чтобы попасть в папку со входящими письмами, необходимо обратиться к соответствующему интерфейсу приложения. Для этого к адресу сайта добавляется /inbox/: https://e.mail.ru/inbox. Интерфейс, который предоставляет доступ к вашим отправленным письмам, находится по адресу https://e.mail.ru/sent/. Получается, что /inbox/ и /sent/ — это и есть эндпоинты.

У `Flask` есть множество продвинутых функций и плагинов, написанных сторонними разработчиками для решения самых разных задач. Однако нам достаточно научиться работать с `Flask` на минимальном уровне, чтобы вывести свою модель как отдельный маленький сервис (микросервис), решающий одну конкретную задачу — к сервису можно обращаться по сети и получать результаты (предсказания модели).

Реализация глобальной структуры сервиса (веб-приложения) и встраивание этой структуры в микросервис — это уже задача других специалистов, например веб-разработчиков.

## DJANGO

В отличие от `Flask`, `Django` нельзя назвать микрофреймворком. Напротив, даже минимальный проект для `Django` обычно генерируется специальными скриптами и включает с десяток файлов и папок.

<img src=DSPROD_12.png width=600>

Такая сложная структура нужна для громоздких приложений, чтобы не путаться в коде. Поэтому Django используют, когда нужно что-то посерьёзней, например целые сайты с базами данных.

ПРЕИМУЩЕСТВА DJANGO:

* Мощный движок для рендера `HTML`-страниц, основанный на шаблонах.

* Собственный `ORM`.

<img src=DSPROD_13.png width=600>

`ОRМ` (`Object Relational Mapping`) — это подход, который позволяет отображать нативные объекты языка (в нашем случае классы `Python`) в таблицы в базе данных. Благодаря этому разработчик может абстрагироваться от написания `SQL`-запросов и «на лету» менять используемую базу данных. В других фреймворках вам придётся использовать сторонние `ORM` (самая популярная — `sqlalchemy`).

* Огромное количество дополнительных плагинов. Один из них практически так же популярен, как и сам `Django` — речь о `Django Rest Framework`. Основываясь на `ORM`, он позволяет в несколько строк реализовывать эндпоинты протокола `RESTful`. Сам `Django Rest Framework` также имеет множество расширений. Благодаря этому написание сложного проекта можно свести в составлению правильной модели данных, установке и настройке подходящих расширений.

## FLASK VS DJANGO

Давайте более подробно рассмотрим характеристики `Flask` и `Django` и узнаем, чем отличаются и чем похожи эти два фреймворка.

В `Django` есть модули для очень широкой функциональности. Чтобы получить к ним доступ, надо просто добавить нужную настройку в конфигурацию. Это отлично перекликается с девизом `Django: “The web framework for perfectionists with deadlines”` («Веб-фреймворк для перфекционистов, которые придерживаются сроков»).

Во `Flask` всё наоборот: сам фреймворк предоставляет только базовую функциональность. «Из коробки» будет доступно направление по адресам эндпойнтов, обработка запросов и ошибок, дебаггер и некоторые другие функции, явно связанные с сервером. Например, чтобы добавить авторизацию, придётся устанавливать стороннюю библиотеку. При этом вы можете выбрать ту, которая лучше подходит текущему проекту, лучше поддерживается или просто более удобна в использовании, а можете вообще не подключать библиотеки.

Flask нужен для оборачивания инференса модели в `API`, когда, например, по поступившему запросу запускается предикт модели. Знание Flask пригодится и на этапе создания прототипа, и на этапе тестирования.

Также некоторые компании строят весь продакшен на `Flask`, а значит, знать его основы будет полезно.

В связи с тем, что `Django` выпускается в релиз вместе со всеми сопутствующими модулями, у него достаточно длинные релиз-циклы. С `Flask` эта проблема менее актуальна: если какой-то функциональности не хватает, часто она есть в какой-то библиотеке. Этот подход жизнеспособен, потому что у `Flask` большое сообщество разработчиков, которые занимаются поддержкой библиотек для этого фреймворка.

`Flask` приобрёл популярность именно благодаря небольшим приложениям, для которых `Django` казался избыточным. Однако сегодня `Flask` используется не только для таких приложений.

Для справки приведём инструментарий, используемый в `Django` и `Flask`, чтобы продемонстировать, как фреймворки отличаются друг от друга по функциональности. Запоминать таблицу не нужно.

Примечание. При разработке на `Flask` список чаще всего не ограничивается перечисленными в таблице библиотеками — здесь собраны только наиболее популярные и известные. Также есть библиотеки, у которых есть специальные расширения для `Flask`. Чаще всего они не обязательны для работы с библиотекой, но могут быть полезны.

| ФУНКЦИОНАЛЬНОСТЬ | DJANGO | FLASK | FLASK EXTENSION |
| - | - | - | - |
| Шаблоны |	Django templates | Jinja2 | | 
| ORM | Django ORM | [sqlalchemy](https://www.sqlalchemy.org/) | [Flask-sqlalchemy](https://flask-sqlalchemy.palletsprojects.com/en/2.x/) |
| Миграции | Django ORM | [Alembic](https://alembic.sqlalchemy.org/en/latest/) | [Flask-alembic](https://flask-alembic.readthedocs.io/en/stable/) |
| Работа с MongoDB | Django ORM | [PyMongo](https://api.mongodb.com/python/current/), [MongoEngine](http://mongoengine.org/) | [Flask-PyMongo](https://flask-pymongo.readthedocs.io/en/latest/), [Flask-MongoEngine](http://docs.mongoengine.org/projects/flask-mongoengine/en/latest/) |
| Авторизация, аутентификация | Django Contrib Auth | [Flask-Login](https://flask-login.readthedocs.io/en/latest/), [Flask-Principal](https://pythonhosted.org/Flask-Principal/) |
| Админка | Django Admin | [Flask-Admin](https://github.com/flask-admin/flask-admin) | |
| Кэширование | Django Cache Framework | [Flask-Caching](https://docs.djangoproject.com/en/2.2/topics/cache/) | |
| REST | [Django REST](https://www.django-rest-framework.org/) | [Flask-RESTful](https://flask-restful.readthedocs.io/en/latest/), [Flask-RESTPlus](https://flask-restplus.readthedocs.io/en/stable/) | |
| Сериализация | [Django REST](https://www.django-rest-framework.org/) | [Marshmallow](https://marshmallow.readthedocs.io/) | [Flask-marshmallow](https://flask-marshmallow.readthedocs.io/en/latest/) |

## FASTAPI

Это относительно молодой фреймворк, однако он уже успел завоевать некоторую популярность. Возможно, через некоторое время он превратится в стандарт индустрии, заменив Flask.

>Примечание. Да-да, именно `Flask`! `FastAPI` тоже позиционируется как легковесный фреймворк. Более того, принцип его использования очень похож на `Flask`. Авторы утверждают, что, вдохновившись `Flask`, они написали более быстрый (за счёт использования более современных протоколов), но при этом не уступающий по функциональности фреймворк.

Основное нововведение в `FastAPI` — интеграция библиотеки `pydantic`, которая позволяет декларативно (Это значит, что вы можете написать класс с несколькими полями, указав их тип с помощью аннотаций, а функционал pydantic позволит вам сериализовывать такие объекты в JSON и обратно без написания дополнительного кода. Благодаря этому вы сможете избавиться от монотонного разбора параметров запроса и даже сделать его более надёжным.) описывать структуры запросов.

Также в `FastAPI` есть поддержка асинхронных функций и реализация парадигмы [Dependency Injection](https://habr.com/ru/post/434380/).

<img src=DSPROD_14.png width=600>

Подробнее об **асинхронных функциях** можно прочитать [здесь](https://habr.com/ru/post/667630/).

## ДРУГИЕ ФРЕЙМВОРКИ

Фреймворк `aiohttp` тоже близок к `Flask`, но основан на асинхронном взаимодействии. `Aiohttp` добавлен в `Python` версии 3.4.

`Tornado` — относительно старый и более общий асинхронный фреймворк для написания сетевых приложений, но также он умеет работать с `HTTP`. По заверениям разработчиков, он очень хорошо держит нагрузку. В 2019 году Tornado занимал третье место по популярности среди разработчиков после `Django` и `Flask`.


# 7. Пишем сервер на Flask <a class="anchor" id=8></a>

[к содержанию](#0)

In [2]:
#pip install flask

In [4]:
from flask import Flask

app = Flask(__name__)

Мы передаём `__name__` при инициализации класса `Flask`, чтобы определить имя, с которым будет использоваться этот модуль. `Flask` использует расположение файла как точку, к которой он привязывает ресурсы.

>Совет. В абсолютном большинстве случаев можно передавать аргумент `__name__` для инициализации класса — приложение будет настроено верно.

Теперь мы можем написать функцию, которая будет обрабатывать запросы, и прикрепить её к какому-то пути (`URI`). Это делается с помощью специального декоратора `route`.

In [5]:
@app.route('/hello')
def hello_func():
    return 'hello!'

Наша функция пока не делает никакой обработки и просто отвечает строкой с приветствием. Нам осталось запустить приложение. Для этого выполним метод `run`, не забыв занести его в стандартный `main`.

In [10]:
if __name__ == '__main__':
    app.run('localhost', 5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5000
Press CTRL+C to quit


>Примечание. В блоке `if __name__ == '__main__'` прописывается код, который не должен выполняться при импорте модуля. Переменная `__name__` — это специальная переменная, которая будет равна `"__main__"`, только если файл запускается как основная программа, и выставляется равной имени модуля при импорте модуля.

Например, если мы захотим импортировать файл server.py как внешний модуль,

In [ ]:
#from server import *

то код, указанный в блоке `if __name__ == '__main__'`, соответствующий запуску сервера, не будет выполнен.

Мы запускаем наш сервис в строке `app.run('localhost', 5000)`, указывая адрес сетевого интерфейса и порт, на котором будет работать сервер. В нашем случае мы работаем на локальной машине по IP-адресу 127.0.0.1, или localhost (то есть доступ к сервису может быть получен только с нашего компьютера), а номер порта, по которому можно отправлять запросы, — 5000.

>Если вы работаете в `Jupyter Notebook`, сохраните весь код в файле `server.py` (не в ноутбуке!) и запустите его из командной строки, выполнив команду `python server.py`.

Вы должны увидеть примерно такой текст:
    * Serving Flask app "server" (lazy loading)

    * Environment: production
    WARNING: This is a development server. Do not use it in a production deployment.
    Use a production WSGI server instead.
    * Debug mode: off
    * Running on http://localhost:5000/ (Press CTRL+C to quit)

In [11]:
# ЗАДАНИЕ 7.3

@app.route('/')
def index():
    return "Test message. The server is running"

In [12]:
# ЗАДАНИЕ 7.4

from flask import Flask
import datetime
app = Flask(__name__)

@app.route('/time')
def current_time():
    return {'time': datetime.datetime.now()}
if __name__ == '__main__':
    app.run('localhost', 5000)

#Для выполнения запроса используется ссылка http://localhost:5000/time.

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5000
Press CTRL+C to quit


## ЧАСТЬ II. ПЕРЕДАЧА ПАРАМЕТРОВ ЗАПРОСА

Давайте немного усложним логику обработки — создадим функцию, которая принимает какой-то параметр и использует его для вычисления результата.

Параметры можно передавать тремя способами:

* через адресную строку;
* через заголовки;
* через тело.

В заголовках обычно передают сервисные параметры. Так как у метода `GET` не бывает тела, остаются только параметры адресной строки.

Вы наверняка видели параметры на сайтах, когда в конец адреса дописывается что-то вроде `?id=10&page=2`. Если поставить в конце адреса вопросительный знак, то после него можно добавлять параметры. Передавать можно любые параметры — они будут перечисляться через & и состоять из имени и значения (`id=10`). Например, если вы будете что-то искать в Google, ваш запрос будет видно в адресной строке в параметре с названием q (от англ. query).

Параметры запроса во `Flask` находятся в специальном объекте request, который нужно импортировать. Параметры адресной строки можно найти в поле `args` этого объекта, где `args` — это словарь.

Давайте немного модифицируем наш код — теперь сервер будет здороваться и обращаться по имени. Для этого занесём параметр name, полученный из `request`, в переменную и воспользуемся этим значением, поставив его в форматированную строку.

In [ ]:
from flask import Flask, request

app = Flask(__name__)
@app.route('/hello')

def hello_func():
    name = request.args.get('name')
    return f'hello {name}!'

if __name__ == '__main__':

    app.run('localhost', 5000)
    
# Перезапустите сервер и зайдите по адресу http://localhost:5000/hello?name=world.

In [14]:
import datetime
now = datetime.datetime.now()
now

datetime.datetime(2023, 10, 2, 14, 5, 53, 550051)

In [ ]:
# ЗАДАНИЕ 7.4

from flask import Flask
import datetime
app = Flask(__name__)

@app.route('/time')
def current_time():
    return {'time': datetime.datetime.now()}
if __name__ == '__main__':
    app.run('localhost', 5000)
    
# Для выполнения запроса используется ссылка http://localhost:5000/time.

## ЧАСТЬ III. POST-ЗАПРОСЫ